In [78]:
%autosave 0
%matplotlib inline

Autosave disabled


In [79]:
import codecs
import pymarc
import sqlite3
import yaml
from collections import defaultdict
import random
import tensorflow

In [80]:
INPUT_MARCXML_FILE = "/home/app/data/input/metadata/marc21.xml"
MARCXML_PARSER_CONF = "/home/app/src/config/parsing_conf.yaml"
INPUT_SAMPLE_SIZE = 10

### Task 1 - Count number of records

In [81]:
#Done - in /src/input/metadata/marcxml_parsing

### Task 2 - Tally frequency of tags

In [82]:
#Done - in /src/input/metadata/marcxml_parsing

### Task 3 - Parse marcxml

In [83]:
marc_records = pymarc.parse_xml_to_array(INPUT_MARCXML_FILE)
try: marc_records = random.sample(marc_records, INPUT_SAMPLE_SIZE)
except: pass

### Task 4 - Map records to dictionary

In [114]:
def load_parser_conf(conf_file):
    with open(conf_file, "r") as stream:
        conf = yaml.load(stream)
    return conf

def get_subfield(marc_field, field, sub_field):
    if not sub_field: return marc_field.value() # for leader
    else: return marc_field[sub_field]

def parse_conf_entry(conf_entry):
    (marc_field, dict_field), = conf_entry.items()
    try: field, sub_field = marc_field.split("$")
    except: field, sub_field = marc_field, None
    return(field, sub_field, dict_field)
    
def map_marc_record(marc_record, conf):
    marc_dict = {}
    for entry in conf:
        field, sub_field, dict_field = parse_conf_entry(entry)
        for marc_field in marc_record.get_fields(field):
            if field == "6??": print("subject")
            elif field == "85*": print("image")
            marc_dict[dict_field] = get_subfield(marc_field, field, sub_field)
    return dict(marc_dict)

def map_marc_records(marc_records, conf_file):
    conf = load_parser_conf(conf_file)
    marc_records_tf = map(lambda record: map_marc_record(record, conf), marc_records)
    return list(marc_records_tf)

marc_dict = map_marc_records(marc_records, MARCXML_PARSER_CONF)

In [109]:
marc_dict[9]

{'collection_id': '.b34164418',
 'date_created': 'ca. 1960.',
 'language_code': 'eng',
 'leader': '130115q1960    xx                i eng d',
 'location_code': 'u-at-we',
 'location_division': None,
 'location_name': '108 Murray Street (Perth, W.A.)',
 'note_general': 'Written on negative: Lotteries Commission.',
 'note_public': None,
 'note_title': 'Lotteries Commission of Western Australia, 110 Murray Street (left) and C&J Colmer Footwear and the Civil Service Club, 108 Murray Street, Perth (right)',
 'note_topical_term': 'Online image.',
 'note_url': 'http://purl.slwa.wa.gov.au/slwa_b3416441_1.png',
 'physical_details': 'acetate, b&w ;',
 'physical_extent': '1 negative :',
 'series_title': 'Western Australian images ;',
 'series_volume': 'BA1521/350.',
 'subject_company_name': 'Civil Service Club'}

In [12]:
def reverse_nested_dict(original_dict):
    for note_url in original_dict["note_url"]:
        image_id = note_url.split("/")[0]

list(map(lambda record: reverse_nested_dict(record), marc_records))

AttributeError: 'set' object has no attribute 'items'